In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import qnmfits
import CCE
from qnmfits.spatial_mapping_functions import *

plt.style.use("stylesheet.mplstyle")

In [2]:
simnums = [
    "0001",
    "0002",
    "0003",
    "0004",
    "0005",
    "0006",
    "0007",
    "0008",
    "0009",
    "0010",
    "0011",
    "0012",
    "0013",
]

sim_options = [("Lev4", "R2"), ("Lev4", "R3"), ("Lev5", "R2"), ("Lev5", "R3")]

In [3]:
start_times = [
    17.0,
    21.0,
    23.0,
    26.0,
    17.0,
    17.0,
    17.0,
    11.0,
    29.0,
    16.0,
    12.0,
    17.0,
    6.0,
]

In [4]:
l_max = 8
n_max = 7

In [5]:
mappings = [
    (lam, mu, n, p)
    for lam in np.arange(2, l_max + 1)
    for mu in np.arange(-lam, lam + 1)
    for n in np.arange(0, n_max + 1)
    for p in (-1, +1)
]

sim_mismatches = {}

In [ ]:
# Compute the spatial mismatch for every QNM in every simulation and save the results

for simnum, t0 in zip(simnums, start_times):
    for option in sim_options:

        lev, radius = option

        name = simnum + "_" + lev + "_" + radius

        print(name)

        sim = CCE.SXS_CCE(simnum, lev=lev, radius=radius)

        mismatch_dict = {}

        for i, map in enumerate(mappings):

            lp, mp, nprime, pp = map

            lpp = abs(mp)
            if lpp < 2:
                lpp = 2

            QNMs = [
                (lam, mp, n, p)
                for lam in np.arange(lpp, l_max + 1)
                for n in np.arange(0, n_max + 1)
                for p in (-1, +1)
            ]

            best_fit = qnmfits.mapping_multimode_ringdown_fit(
                sim.times,
                sim.h,
                modes=QNMs.copy(),
                Mf=sim.Mf,
                chif=sim.chif_mag,
                t0=t0,
                mapping_modes=[map],
                spherical_modes=[(lam, mp) for lam in np.arange(lpp, l_max + 1)],
            )

            sm, arg, _ = spatial_mismatch_linear(best_fit, map, sim.chif_mag, l_max)

            mismatch_dict[map] = sm

        sim_mismatches[name] = mismatch_dict

In [7]:
with open("sim_mismatches_new.pickle", "wb") as file:
    pickle.dump(sim_mismatches, file)